In [ ]:
#importing reqired libraries
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<li>Removed the not relevant column</li>
<li>Replace missing values with mean (for numerical columns) or mode (for categorical columns)
<li>Transformed the categorical data to binary features
<li>Preprocessed the year columns using a function below
<li>standardized the data



In [ ]:
def normalize_emp_length(value):
       if pd.isna(value):
           return 0  # or any suitable numeric value
       if isinstance(value, str):
           if value == '< 1 year':
               return 0
           elif value == '10+ years':
               return 10
           else:
               try:
                   return int(value.split()[0]) # Extract numeric part
               except ValueError:
                   print("*\n"*10)
                   print(f"Unexpected value in emp_length: {value}")
                   return 0  # or any suitable numeric value for unexpected cases
       else:
           return ""  # Assuming it's already a numeric value

def check_nulls_in_df(df):
  columns_with_nulls = []
  for column in df.columns:
    if df[column].isnull().any():
      columns_with_nulls.append(column)
  return columns_with_nulls

def pre_process_data(data):

  data = data[data['loan_status'].isin(['Fully Paid', 'Charged Off'])].copy()
  # print(train_data[train_data['loan_status']=='current'])
  data['int_rate'] = data['int_rate'].str.replace('%', '').astype(float)
  data['term'] = data['term'].str.replace('months', '').astype(float)
  data['revol_util'] = data['revol_util'].str.replace('%', '').astype(float)

  data['emp_length'] = data['emp_length'].apply(normalize_emp_length)

  # Replace missing values with mean (for numerical columns) or mode (for categorical columns)
  data.fillna(data.mode().iloc[0], inplace=True)
  # train_data.fillna(train_data.mean(), inplace=True)

  # train_data.fillna()

  # print("train data:",data.info())


  # Check for the existence of the columns before applying pd.get_dummies
  columns_to_dummies = ['emp_length','grade', 'sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state', 'application_type']
  for column in columns_to_dummies:
      if column not in data.columns:
          # print(f"Column '{column}' not found in train_data")
          # Handle the missing column (e.g., skip, create a new column, or raise an error)
          # Here, we'll skip creating dummies for missing columns
          columns_to_dummies.remove(column)

  # Apply pd.get_dummies only to the existing columns
  data = pd.get_dummies(data, columns=columns_to_dummies)


  # train_data = pd.get_dummies(train_data, columns=['grade', 'sub_grade',  'verification_status', 'purpose', 'addr_state', 'application_type'])


  num_features = ['loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'revol_bal', 'total_acc', 'out_prncp', 'total_pymnt', 'total_rec_prncp', 'total_rec_int']

  data['issue_d'] = pd.to_datetime(data['issue_d'], format='%b-%y', errors='coerce')
  data['issue_d_year'] = data['issue_d'].dt.year
  data['issue_d_month'] = data['issue_d'].dt.month
  data = data.drop(columns=['issue_d']) # Remove original 'issue_d' column


  # Convert 'earliest_cr_line' to datetime and extract numerical features if present
  data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'], format='%b-%y',errors='coerce')
  data['earliest_cr_line_year'] = data['earliest_cr_line'].dt.year
  data['earliest_cr_line_month'] = data['earliest_cr_line'].dt.month
  data = data.drop(columns=['earliest_cr_line']) # Remove original 'earliest_cr_line' column

  data['last_pymnt_d'] = pd.to_datetime(data['last_pymnt_d'], format='%b-%y',errors='coerce')
  data['last_pymnt_d_year'] = data['last_pymnt_d'].dt.year
  data['last_pymnt_d_month'] = data['last_pymnt_d'].dt.month
  data = data.drop(columns=['last_pymnt_d'])

  data.fillna(data.mode().iloc[0], inplace=True)

  # Standardize numerical features
  scaler = StandardScaler()
  data[num_features] = scaler.fit_transform(data[num_features])




  # print("train data:",data.info())

  # print(data.head())
  return data
  pass


In [ ]:
train_url = "/content/drive/My Drive/Colab Notebooks/loan_train.csv"
test_url = "/content/drive/My Drive/Colab Notebooks/loan_test.csv"

# Load the datasets
train_data = pd.read_csv(train_url)
test_data = pd.read_csv(test_url)

# Preprocess both datasets first
train_data = pre_process_data(train_data)
test_data = pre_process_data(test_data)

# Check for nulls and drop columns consistently
null_columns = check_nulls_in_df(train_data) # Check nulls in train data
null_columns_to_drop = [col for col in null_columns if col in train_data.columns]

# Drop columns with nulls from both datasets
train_data = train_data.drop(columns=null_columns_to_drop)
test_data = test_data.drop(columns=null_columns_to_drop)

# Now split the data into X_train, y_train, X_test, y_test
X_train = train_data.drop(columns=['last_credit_pull_d','last_pymnt_amnt','initial_list_status','title','pymnt_plan','loan_status', 'id', 'member_id', 'desc', 'url', 'zip_code', 'emp_title'])
y_train = train_data['loan_status']

X_test = test_data.drop(columns=['last_credit_pull_d','last_pymnt_amnt','initial_list_status','title','pymnt_plan','loan_status', 'id', 'member_id', 'desc', 'url', 'zip_code', 'emp_title'])
y_test = test_data['loan_status']


common_columns = X_train.columns.intersection(X_test.columns)

# print(common_columns)
X_train=X_train[common_columns]
X_test=X_test[common_columns]


<ipython-input-29-d3ab15ec80b2>:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(data.mode().iloc[0], inplace=True)
<ipython-input-29-d3ab15ec80b2>:37: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.fillna(data.mode().iloc[0], inplace=True)


In [ ]:
#training data cannot have a string for training here
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train the gradient boosting model
gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.2, max_depth=7)
gb_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = gb_clf.predict(X_test)

# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

 Charged Off       1.00      0.98      0.99      2153
  Fully Paid       1.00      1.00      1.00     12123

    accuracy                           1.00     14276
   macro avg       1.00      0.99      0.99     14276
weighted avg       1.00      1.00      1.00     14276

Accuracy: 0.9972681423367891


In [ ]:
# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10]
}

# param_grid = {
#     'n_estimators': [100 ],
#     'learning_rate': [ 0.2],
#     'max_depth': [3],
#     'min_samples_split': [2, 5]
# }


scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score)
}
# Initialize GradientBoostingClassifier and GridSearchCV
gb_clf = GradientBoostingClassifier()
grid_search = GridSearchCV(
    estimator=gb_clf,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    refit='accuracy',  # Use accuracy to select the best model
    n_jobs=-1,
    verbose=2
)
# grid_search = GridSearchCV(estimator=gb_clf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)

# Fit model
grid_search.fit(X_train, y_train)

# Output best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)




In [ ]:
# # Define parameter grid
# # param_grid = {
# #     'n_estimators': [100, 200, 300],
# #     'learning_rate': [0.1, 0.2, 0.3],
# #     'max_depth': [3, 5, 7],
# #     'min_samples_split': [2, 5, 10]
# # }

# param_grid = {
#     'n_estimators': [100 ],
#     'learning_rate': [ 0.2],
#     'max_depth': [3],
#     'min_samples_split': [2, 5]
# }

# # Update the grid search to include multiple scoring metrics
# scoring = {
#     'precision': make_scorer(precision_score, zero_division=1),  # Handle cases with zero division
#     'recall': make_scorer(recall_score, zero_division=1)
# }

# grid_search = GridSearchCV(
#     estimator=gb_clf,
#     param_grid=param_grid,
#     cv=5,
#     scoring=scoring,
#     refit='precision',  # You can refit based on precision or recall, or 'accuracy'
#     n_jobs=-1,
#     verbose=2
# )

# # Fit the grid search
# grid_search.fit(X_train, y_train)

# # Print precision and recall for each parameter set
# print("\nPrecision and Recall for each model:")
# for i in range(len(grid_search.cv_results_['params'])):
#     params = grid_search.cv_results_['params'][i]
#     mean_precision = grid_search.cv_results_['mean_test_precision'][i]
#     mean_recall = grid_search.cv_results_['mean_test_recall'][i]
#     print(f"Params: {params}")
#     print(f"Precision: {mean_precision:.4f}, Recall: {mean_recall:.4f}\n")

**Increase in the number of trees:**
<li>Accuracy increases as the trees increases
<li>Accuracy increases upto a certain extent rapidly and then achieves lesser accuracy increase as we increase the number of trees.
<li>Model might overfit after certain point


In [ ]:
dt_clf = DecisionTreeClassifier(criterion='entropy', max_depth=None)  # No depth limit
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)

y_test_numeric = y_test.map({'Charged Off': 0, 'Fully Paid': 1})
y_pred_dt_numeric = pd.Series(y_pred_dt).map({'Charged Off': 0, 'Fully Paid': 1})

accuracy_dt = accuracy_score(y_test_numeric, y_pred_dt_numeric)
precision_dt = precision_score(y_test_numeric, y_pred_dt_numeric)
recall_dt = recall_score(y_test_numeric, y_pred_dt_numeric)


print("Decision Tree Classifier Performance:")
print(f"Accuracy: {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")

Decision Tree Classifier Performance:
Accuracy: 0.9970
Precision: 0.9977
Recall: 0.9988
